In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 79.7 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cud

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pickle
import os
import random
from torch.utils.data import Dataset, DataLoader
from typing import Dict, List, Tuple, Optional
import json
from pathlib import Path
from collections import defaultdict
import re
from sklearn.metrics.pairwise import cosine_similarity
import joblib

class MusicContrastiveDataset(Dataset):
    """
    Ultra-fast dataset for contrastive learning with lazy validation and caching
    """
    def __init__(self,
                 symbolic_dir: str,
                 audio_dir: str,
                 metadata_csv: str,
                 negative_ratio: float = 0.6,
                 hard_negative_ratio: float = 0.4,
                 cache_file: str = "file_validation_cache.pkl"):

        self.symbolic_dir = Path(symbolic_dir)
        self.audio_dir = Path(audio_dir)
        self.metadata_csv = metadata_csv
        self.negative_ratio = negative_ratio
        self.hard_negative_ratio = hard_negative_ratio
        self.cache_file = cache_file

        # Load or create validation cache
        self.validation_cache = self._load_or_create_cache()

        # Load metadata first
        self.metadata_df = self._load_metadata_csv()

        # Fast file loading (no validation yet)
        self.symbolic_files = self._load_file_list_fast(self.symbolic_dir, "symbolic")
        self.audio_files = self._load_file_list_fast(self.audio_dir, "audio")

        # Create enhanced mappings with metadata
        self.segment_mapping = self._create_enhanced_segment_mapping()
        self.pairs = self._create_strategic_pairs()

    def _load_or_create_cache(self) -> Dict:
        """Load existing validation cache or create empty one"""
        cache_path = str(self.symbolic_dir.parent / self.cache_file)
        if os.path.exists(cache_path):
            try:
                with open(cache_path, 'rb') as f:
                    cache = pickle.load(f)
                print(f"Loaded validation cache with {len(cache)} entries")
                return cache
            except Exception as e:
                print(f"Error loading cache: {e}")

        print("Creating new validation cache")
        return {}

    def _save_cache(self):
        """Save validation cache"""
        try:
            cache_path = str(self.symbolic_dir.parent / self.cache_file)
            with open(cache_path, 'wb') as f:
                pickle.dump(self.validation_cache, f)
            print(f"Saved validation cache with {len(self.validation_cache)} entries")
        except Exception as e:
            print(f"Error saving cache: {e}")

    def _quick_file_check(self, file_path: str, expected_dim: int) -> Tuple[bool, Optional[int], str]:
        """
        Ultra-fast file validation using multiple heuristics
        Returns: (is_valid, dimension, reason)
        """
        # Check cache first
        if file_path in self.validation_cache:
            cached = self.validation_cache[file_path]
            return cached['valid'], cached['dim'], cached['reason']

        # Fast checks without loading file content
        try:
            # Check 1: File exists and has reasonable size
            if not os.path.exists(file_path):
                result = (False, None, "file_not_found")
                self.validation_cache[file_path] = {'valid': False, 'dim': None, 'reason': "file_not_found"}
                return result

            file_size = os.path.getsize(file_path)

            # Check 2: File size heuristics
            if file_size < 100:  # Too small to contain meaningful data
                result = (False, None, "file_too_small")
                self.validation_cache[file_path] = {'valid': False, 'dim': None, 'reason': "file_too_small"}
                return result

            # Check 3: File size consistency with expected dimensions
            if expected_dim:
                # Rough size estimation: float32 (4 bytes) * expected_dim + numpy overhead
                min_expected_size = expected_dim * 4 + 100  # +100 for numpy header
                max_expected_size = expected_dim * 4 * 100   # Allow up to 100 time steps

                if file_size < min_expected_size:
                    result = (False, expected_dim, "size_too_small_for_dimension")
                    self.validation_cache[file_path] = {'valid': False, 'dim': expected_dim, 'reason': "size_too_small_for_dimension"}
                    return result

            # If all fast checks pass, assume valid (validate during actual loading)
            result = (True, expected_dim, "passed_fast_checks")
            self.validation_cache[file_path] = {'valid': True, 'dim': expected_dim, 'reason': "passed_fast_checks"}
            return result

        except Exception as e:
            result = (False, None, f"error: {str(e)}")
            self.validation_cache[file_path] = {'valid': False, 'dim': None, 'reason': f"error: {str(e)}"}
            return result
    def _load_metadata_csv(self) -> pd.DataFrame:
        """Load the MusicNet metadata CSV file"""
        try:
            df = pd.read_csv(self.metadata_csv)
            print(f"Loaded metadata for {len(df)} pieces")
            print(f"Columns: {list(df.columns)}")

            # Group by composer for strategic pairing
            self.composer_groups = df.groupby('composer')['id'].apply(list).to_dict()
            print(f"Found {len(self.composer_groups)} composers")

            # Create era mapping based on composer (simplified)
            self.era_mapping = self._create_era_mapping(df)

            return df
        except Exception as e:
            print(f"Error loading metadata CSV: {e}")
            return pd.DataFrame()

    def _create_era_mapping(self, df: pd.DataFrame) -> Dict:
        """Create era mapping for composers based on actual dataset"""
        era_mapping = {}

        # Based on the actual composers in your dataset
        baroque_composers = ['Bach']
        classical_composers = ['Mozart', 'Haydn', 'Beethoven']
        romantic_composers = ['Brahms', 'Schubert', 'Dvorak']
        impressionist_composers = ['Ravel', 'Faure']
        modern_composers = ['Cambini']  # 18th-19th century

        for composer in df['composer'].unique():
            composer_str = str(composer).lower()
            if any(name.lower() in composer_str for name in baroque_composers):
                era_mapping[composer] = 'Baroque'
            elif any(name.lower() in composer_str for name in classical_composers):
                era_mapping[composer] = 'Classical'
            elif any(name.lower() in composer_str for name in romantic_composers):
                era_mapping[composer] = 'Romantic'
            elif any(name.lower() in composer_str for name in impressionist_composers):
                era_mapping[composer] = 'Impressionist'
            elif any(name.lower() in composer_str for name in modern_composers):
                era_mapping[composer] = 'Classical-Modern'
            else:
                era_mapping[composer] = 'Other'

        return era_mapping

    def _load_file_list_fast(self, directory: Path, file_type: str) -> Dict[str, List[str]]:
        """Ultra-fast file list loading with lazy validation"""
        files_by_id = {}
        expected_dim = 768 if file_type == "symbolic" else 6144

        print(f"Fast-loading {file_type} files from: {directory}")

        npy_files = list(directory.glob("*.npy"))
        print(f"Found {len(npy_files)} files")

        valid_count = 0
        invalid_count = 0

        for file_path in npy_files:
            # Quick validation without loading file content
            is_valid, actual_dim, reason = self._quick_file_check(str(file_path), expected_dim)

            if not is_valid:
                invalid_count += 1
                if invalid_count <= 5:  # Only show first few invalid files
                    print(f"Skipping invalid {file_type} file {file_path.name}: {reason}")
                continue

            # Extract ID from filename
            filename = file_path.stem
            if '_' in filename:
                parts = filename.split('_')
                for part in parts:
                    if part.isdigit():
                        data_id = part
                        break
                else:
                    data_id = parts[0]
            else:
                numbers = re.findall(r'\d+', filename)
                data_id = numbers[0] if numbers else filename

            if data_id not in files_by_id:
                files_by_id[data_id] = []
            files_by_id[data_id].append(str(file_path))
            valid_count += 1

            # Save cache periodically to avoid losing progress
            if (valid_count + invalid_count) % 1000 == 0:
                self._save_cache()
                print(f"  Processed {valid_count + invalid_count} files...")

        # Save final cache
        self._save_cache()

        print(f"Fast validation complete: {valid_count} valid, {invalid_count} invalid")
        print(f"Organized into {len(files_by_id)} ID groups")
        return files_by_id

    def _create_enhanced_segment_mapping(self) -> Dict:
        """Create enhanced mapping with metadata information"""
        mapping = {}

        # Find common IDs between symbolic, audio, and metadata
        metadata_ids = set(str(id_) for id_ in self.metadata_df['id'].tolist())
        symbolic_ids = set(self.symbolic_files.keys())
        audio_ids = set(self.audio_files.keys())

        common_ids = metadata_ids & symbolic_ids & audio_ids
        print(f"Common IDs across all sources: {len(common_ids)}")

        if len(common_ids) == 0:
            print("ERROR: No common IDs found between symbolic, audio, and metadata!")
            print(f"Metadata IDs sample: {list(metadata_ids)[:5]}")
            print(f"Symbolic IDs sample: {list(symbolic_ids)[:5]}")
            print(f"Audio IDs sample: {list(audio_ids)[:5]}")

        for data_id in common_ids:
            # Get metadata for this piece
            metadata_row = self.metadata_df[self.metadata_df['id'] == int(data_id)].iloc[0]

            mapping[data_id] = {
                'symbolic': self.symbolic_files[data_id],
                'audio': self.audio_files[data_id],
                'composer': metadata_row['composer'],
                'composition': metadata_row['composition'],
                'movement': metadata_row.get('movement', ''),
                'ensemble': metadata_row.get('ensemble', ''),
                'seconds': metadata_row.get('seconds', 0),
                'era': self.era_mapping.get(metadata_row['composer'], 'Other')
            }

        print(f"Created enhanced mapping for {len(mapping)} pieces")
        return mapping

    def _create_strategic_pairs(self) -> List[Dict]:
        """Create strategic positive and negative pairs using metadata"""
        pairs = []
        data_ids = list(self.segment_mapping.keys())

        if len(data_ids) == 0:
            print("ERROR: No data IDs available for creating pairs!")
            return pairs

        print(f"Creating strategic pairs from {len(data_ids)} pieces...")

        # Print composer and era distribution for debugging
        composer_counts = defaultdict(int)
        era_counts = defaultdict(int)
        for data_id in data_ids:
            composer_counts[self.segment_mapping[data_id]['composer']] += 1
            era_counts[self.segment_mapping[data_id]['era']] += 1

        print("Composer distribution:", dict(composer_counts))
        print("Era distribution:", dict(era_counts))

        # 1. CREATE POSITIVE PAIRS
        positive_pairs = self._create_positive_pairs(data_ids)
        pairs.extend(positive_pairs)

        # 2. CREATE NEGATIVE PAIRS
        negative_pairs = self._create_negative_pairs(data_ids, len(positive_pairs))
        pairs.extend(negative_pairs)

        print(f"Created {len(pairs)} total pairs:")
        print(f"  - Positive: {len(positive_pairs)}")
        print(f"  - Negative: {len(negative_pairs)}")

        return pairs

    def _create_positive_pairs(self, data_ids: List[str]) -> List[Dict]:
        """Create strategic positive pairs"""
        positive_pairs = []

        for data_id in data_ids:
            segments = self.segment_mapping[data_id]
            symbolic_segments = segments['symbolic']
            audio_segments = segments['audio']

            # Type 1: Direct matches (same segment)
            min_segments = min(len(symbolic_segments), len(audio_segments))
            for i in range(min_segments):
                positive_pairs.append({
                    'symbolic_path': symbolic_segments[i],
                    'audio_path': audio_segments[i],
                    'label': 1,
                    'data_id': data_id,
                    'pair_type': 'positive_direct',
                    'composer': segments['composer'],
                    'composition': segments['composition']
                })

            # Type 2: Overlapping segments (same recording, different segments)
            if min_segments > 1:
                for i in range(min_segments - 1):
                    for j in range(i + 1, min(i + 3, min_segments)):  # Nearby segments
                        positive_pairs.append({
                            'symbolic_path': symbolic_segments[i],
                            'audio_path': audio_segments[j],
                            'label': 1,
                            'data_id': data_id,
                            'pair_type': 'positive_nearby',
                            'composer': segments['composer'],
                            'composition': segments['composition']
                        })

        print(f"Created {len(positive_pairs)} positive pairs")
        return positive_pairs

    def _create_negative_pairs(self, data_ids: List[str], num_positives: int) -> List[Dict]:
        """Create strategic negative pairs (hard negatives) based on actual dataset composers"""
        negative_pairs = []
        num_negatives = int(num_positives * self.negative_ratio / (1 - self.negative_ratio))

        # Calculate distribution of negative types
        hard_negatives = int(num_negatives * self.hard_negative_ratio)
        medium_negatives = int(num_negatives * 0.3)
        easy_negatives = num_negatives - hard_negatives - medium_negatives

        print(f"Creating {num_negatives} negative pairs:")
        print(f"  - Hard (same composer): {hard_negatives}")
        print(f"  - Medium (same era): {medium_negatives}")
        print(f"  - Easy (different era): {easy_negatives}")

        # Group pieces by composer and era
        composer_groups = defaultdict(list)
        era_groups = defaultdict(list)

        for data_id in data_ids:
            composer = self.segment_mapping[data_id]['composer']
            era = self.segment_mapping[data_id]['era']
            composer_groups[composer].append(data_id)
            era_groups[era].append(data_id)

        # Type 1: Hard negatives - Same composer, different compositions
        hard_created = 0
        for composer, pieces in composer_groups.items():
            if len(pieces) > 1 and hard_created < hard_negatives:
                for _ in range(min(len(pieces) // 2, hard_negatives - hard_created)):
                    if len(pieces) >= 2:
                        id1, id2 = random.sample(pieces, 2)
                        # Ensure different compositions
                        if (self.segment_mapping[id1]['composition'] !=
                            self.segment_mapping[id2]['composition']):
                            negative_pairs.append(self._create_negative_pair(id1, id2, 'hard_same_composer'))
                            hard_created += 1

        # Type 2: Medium negatives - Same era, different composers
        medium_created = 0
        for era, pieces in era_groups.items():
            if len(pieces) > 1 and medium_created < medium_negatives:
                era_composers = set(self.segment_mapping[p]['composer'] for p in pieces)
                if len(era_composers) > 1:
                    for _ in range(min(len(pieces) // 3, medium_negatives - medium_created)):
                        id1, id2 = random.sample(pieces, 2)
                        if self.segment_mapping[id1]['composer'] != self.segment_mapping[id2]['composer']:
                            negative_pairs.append(self._create_negative_pair(id1, id2, 'medium_same_era'))
                            medium_created += 1

        # Type 3: Easy negatives - Different composers, different eras
        different_era_pairs = []
        for i, id1 in enumerate(data_ids):
            for id2 in data_ids[i+1:]:
                if (self.segment_mapping[id1]['composer'] != self.segment_mapping[id2]['composer'] and
                    self.segment_mapping[id1]['era'] != self.segment_mapping[id2]['era']):
                    different_era_pairs.append((id1, id2))

        for _ in range(min(easy_negatives, len(different_era_pairs))):
            id1, id2 = random.choice(different_era_pairs)
            negative_pairs.append(self._create_negative_pair(id1, id2, 'easy_different_era'))

        print(f"Actually created: {hard_created} hard, {medium_created} medium, {easy_negatives} easy")
        return negative_pairs

    def _create_negative_pair(self, id1: str, id2: str, pair_subtype: str) -> Dict:
        """Helper to create a negative pair"""
        symbolic_seg = random.choice(self.segment_mapping[id1]['symbolic'])
        audio_seg = random.choice(self.segment_mapping[id2]['audio'])

        return {
            'symbolic_path': symbolic_seg,
            'audio_path': audio_seg,
            'label': 0,
            'data_id_symbolic': id1,
            'data_id_audio': id2,
            'pair_type': f'negative_{pair_subtype}',
            'composer_symbolic': self.segment_mapping[id1]['composer'],
            'composer_audio': self.segment_mapping[id2]['composer']
        }

    def _load_features(self, file_path: str, expected_dim: int) -> torch.Tensor:
        """
        Lazy feature loading with caching of problematic files
        Only validates dimension on first load, then caches the result
        """
        try:
            # Check if this file was marked as invalid in cache
            if file_path in self.validation_cache:
                cached = self.validation_cache[file_path]
                if not cached['valid']:
                    # Return zero tensor for invalid files
                    return torch.zeros(1, expected_dim)

            features = np.load(file_path)

            # Handle different numpy formats
            if features.dtype == object:
                if features.shape == ():
                    data = features.item()
                    if isinstance(data, dict):
                        if 'features' in data:
                            features = data['features']
                        elif 'embeddings' in data:
                            features = data['embeddings']
                        else:
                            for key, value in data.items():
                                if isinstance(value, (np.ndarray, list)):
                                    features = value
                                    break
                    else:
                        features = data
                else:
                    features = features[0]

            if isinstance(features, list):
                features = np.array(features)

            if not isinstance(features, np.ndarray):
                features = np.array(features)

            # Check for empty arrays
            if features.size == 0:
                # Mark as invalid in cache
                self.validation_cache[file_path] = {'valid': False, 'dim': None, 'reason': 'empty_array'}
                return torch.zeros(1, expected_dim)

            features = torch.tensor(features, dtype=torch.float32)

            if features.dim() == 1:
                features = features.unsqueeze(0)

            if features.dim() > 2:
                features = features.view(features.size(0), -1)

            # Validate dimensions and fix if needed
            actual_dim = features.shape[-1]
            if actual_dim != expected_dim:
                if actual_dim < expected_dim:
                    # Pad with zeros
                    padding = torch.zeros(features.shape[0], expected_dim - actual_dim)
                    features = torch.cat([features, padding], dim=1)
                else:
                    # Truncate
                    features = features[:, :expected_dim]

                # Update cache with corrected info
                self.validation_cache[file_path] = {'valid': True, 'dim': expected_dim, 'reason': 'dimension_corrected'}

            return features

        except Exception as e:
            # Mark as invalid in cache
            self.validation_cache[file_path] = {'valid': False, 'dim': None, 'reason': f'load_error: {str(e)}'}
            print(f"Error loading {file_path}: {e}")
            return torch.zeros(1, expected_dim)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]

        # Load features with proper dimension expectations
        symbolic_features = self._load_features(pair['symbolic_path'], expected_dim=768)
        audio_features = self._load_features(pair['audio_path'], expected_dim=6144)
        label = torch.tensor(pair['label'], dtype=torch.float32)

        return {
            'symbolic': symbolic_features,
            'audio': audio_features,
            'label': label,
            'pair_type': pair['pair_type']
        }


# Custom collate function to handle any remaining inconsistencies
def custom_collate_fn(batch):
    """Custom collate function that ensures consistent tensor shapes"""
    try:
        # Separate the components
        symbolic_list = []
        audio_list = []
        labels = []
        pair_types = []

        for item in batch:
            symbolic_list.append(item['symbolic'])
            audio_list.append(item['audio'])
            labels.append(item['label'])
            pair_types.append(item['pair_type'])

        # Stack tensors with error handling
        try:
            symbolic_batch = torch.stack(symbolic_list, dim=0)
        except RuntimeError as e:
            print(f"Error stacking symbolic features: {e}")
            # Fallback: ensure all have same shape
            min_dim = min(t.shape[-1] for t in symbolic_list)
            symbolic_list = [t[:, :min_dim] for t in symbolic_list]
            symbolic_batch = torch.stack(symbolic_list, dim=0)

        try:
            audio_batch = torch.stack(audio_list, dim=0)
        except RuntimeError as e:
            print(f"Error stacking audio features: {e}")
            # Fallback: ensure all have same shape
            min_dim = min(t.shape[-1] for t in audio_list)
            audio_list = [t[:, :min_dim] for t in audio_list]
            audio_batch = torch.stack(audio_list, dim=0)

        labels_batch = torch.stack(labels, dim=0)

        return {
            'symbolic': symbolic_batch,
            'audio': audio_batch,
            'label': labels_batch,
            'pair_type': pair_types
        }

    except Exception as e:
        print(f"Error in custom_collate_fn: {e}")
        # Return a minimal batch to avoid complete failure
        batch_size = len(batch)
        return {
            'symbolic': torch.zeros(batch_size, 1, 768),
            'audio': torch.zeros(batch_size, 1, 6144),
            'label': torch.zeros(batch_size),
            'pair_type': ['error'] * batch_size
        }


class UnifiedContrastiveModel(nn.Module):
    """
    Enhanced model that outputs unified embeddings
    [Audio Features] + [MIDI Features] → Contrastive Model → [One Unified Embedding]
    """
    def __init__(self, symbolic_dim=768, audio_dim=6144, latent_dim=512):
        super().__init__()

        # Individual encoders
        self.symbolic_encoder = self._build_encoder(symbolic_dim, latent_dim // 2)
        self.audio_encoder = self._build_encoder(audio_dim, latent_dim // 2)

        # Fusion layer for unified embedding
        self.fusion_layer = nn.Sequential(
            nn.Linear(latent_dim, latent_dim),
            nn.BatchNorm1d(latent_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(latent_dim, latent_dim)
        )

        # Temperature parameter for contrastive learning
        self.temperature = nn.Parameter(torch.tensor(0.07))

    def _build_encoder(self, input_dim: int, output_dim: int) -> nn.Module:
        """Build encoder network"""
        hidden_dim = max(512, input_dim // 4)
        return nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, symbolic_features, audio_features):
        # Ensure proper shapes
        if symbolic_features.dim() == 3:
            symbolic_features = symbolic_features.squeeze(1)  # [batch, 1, 768] -> [batch, 768]
        if audio_features.dim() == 3:
            audio_features = audio_features.squeeze(1)        # [batch, 1, 6144] -> [batch, 6144]

        # Encode individual modalities
        symbolic_emb = self.symbolic_encoder(symbolic_features)
        audio_emb = self.audio_encoder(audio_features)

        # Concatenate and create unified embedding
        combined = torch.cat([symbolic_emb, audio_emb], dim=1)
        unified_embedding = self.fusion_layer(combined)

        # Normalize the unified embedding
        unified_embedding = F.normalize(unified_embedding, p=2, dim=1)

        return unified_embedding

    def get_unified_embedding(self, symbolic_features, audio_features):
        """Get the unified embedding for a pair"""
        return self.forward(symbolic_features, audio_features)


class InfoNCELoss(nn.Module):
    """InfoNCE loss for contrastive learning with unified embeddings"""
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature

    def forward(self, embeddings, labels):
        # embeddings: [batch_size, embedding_dim]
        # labels: [batch_size] (1 for positive, 0 for negative)

        batch_size = embeddings.shape[0]

        # Create similarity matrix
        sim_matrix = torch.matmul(embeddings, embeddings.T) / self.temperature

        # Create positive mask
        pos_mask = labels.unsqueeze(1) * labels.unsqueeze(0)
        neg_mask = 1 - pos_mask

        # Compute InfoNCE loss
        pos_sim = sim_matrix * pos_mask
        neg_sim = sim_matrix * neg_mask

        # For each positive pair, compute loss
        pos_pairs = torch.where(pos_mask == 1)
        if len(pos_pairs[0]) == 0:
            return torch.tensor(0.0, device=embeddings.device)

        loss = 0
        for i, j in zip(pos_pairs[0], pos_pairs[1]):
            if i != j:  # Don't include self-similarity
                numerator = torch.exp(sim_matrix[i, j])
                denominator = torch.sum(torch.exp(sim_matrix[i, :]) * neg_mask[i, :]) + numerator
                loss += -torch.log(numerator / (denominator + 1e-8))  # Add epsilon for stability

        return loss / max(len(pos_pairs[0]), 1)


class ReferenceBase:
    """Reference database for plagiarism detection"""
    def __init__(self, save_dir: str = "/content/drive/MyDrive/MuseGuard"):
        self.embeddings = {}
        self.metadata = {}
        self.embedding_matrix = None
        self.ids = []
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)

    def add_reference(self, piece_id: str, embedding: np.ndarray, metadata: Dict):
        """Add a reference piece to the database"""
        self.embeddings[piece_id] = embedding
        self.metadata[piece_id] = metadata

    def build_index(self):
        """Build searchable index"""
        if not self.embeddings:
            return

        self.ids = list(self.embeddings.keys())
        self.embedding_matrix = np.vstack([self.embeddings[id_] for id_ in self.ids])
        print(f"Built reference index with {len(self.ids)} pieces")

    def search_similar(self, query_embedding: np.ndarray, top_k: int = 10, threshold: float = 0.7):
        """Search for similar pieces"""
        if self.embedding_matrix is None:
            self.build_index()

        # Compute similarities
        similarities = cosine_similarity([query_embedding], self.embedding_matrix)[0]

        # Get top-k similar pieces
        top_indices = np.argsort(similarities)[::-1][:top_k]

        results = []
        for idx in top_indices:
            similarity = similarities[idx]
            if similarity >= threshold:
                piece_id = self.ids[idx]
                results.append({
                    'piece_id': piece_id,
                    'similarity': similarity,
                    'metadata': self.metadata[piece_id]
                })

        return results

    def save(self, filename: str = "music_reference_base.pkl"):
        """Save reference base to specified directory"""
        filepath = self.save_dir / filename
        try:
            data = {
                'embeddings': self.embeddings,
                'metadata': self.metadata,
                'embedding_matrix': self.embedding_matrix,
                'ids': self.ids
            }
            joblib.dump(data, filepath)
            print(f"Reference base saved to {filepath}")
        except Exception as e:
            print(f"Error saving reference base: {e}")

    def load(self, filename: str = "music_reference_base.pkl"):
        """Load reference base from specified directory"""
        filepath = self.save_dir / filename
        try:
            data = joblib.load(filepath)
            self.embeddings = data['embeddings']
            self.metadata = data['metadata']
            self.embedding_matrix = data['embedding_matrix']
            self.ids = data['ids']
            print(f"Reference base loaded from {filepath}")
        except Exception as e:
            print(f"Error loading reference base: {e}")


class EnhancedMusicPlagiarismDetector:
    """Enhanced plagiarism detector with unified embeddings and reference base"""

    def __init__(self,
                 symbolic_dir: str,
                 audio_dir: str,
                 metadata_csv: str,
                 save_dir: str = "/content/drive/MyDrive/MuseGuard",
                 device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):

        self.device = device
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(exist_ok=True)

        self.model = UnifiedContrastiveModel().to(device)
        self.criterion = InfoNCELoss()
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-4, weight_decay=1e-4)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=100, eta_min=1e-6
        )

        # Create enhanced dataset
        self.dataset = MusicContrastiveDataset(
            symbolic_dir=symbolic_dir,
            audio_dir=audio_dir,
            metadata_csv=metadata_csv
        )

        # Create data loaders with custom collate function
        self.train_loader = DataLoader(
            self.dataset,
            batch_size=16,  # Reduced batch size for stability
            shuffle=True,
            num_workers=2,  # Reduced workers to avoid multiprocessing issues
            pin_memory=True,
            drop_last=True,
            collate_fn=custom_collate_fn  # Use custom collate function
        )

        # Initialize reference base with save directory
        self.reference_base = ReferenceBase(save_dir=save_dir)

        # Training statistics
        self.training_stats = {
            'epoch_losses': [],
            'best_loss': float('inf'),
            'epochs_trained': 0
        }

    def save_training_state(self, epoch: int, loss: float, is_best: bool = False):
        """Save complete training state including model, optimizer, and reference base"""
        try:
            # Prepare checkpoint data
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'loss': loss,
                'training_stats': self.training_stats,
                'best_loss': self.training_stats['best_loss']
            }

            # Save checkpoint
            checkpoint_path = self.save_dir / f"checkpoint_epoch_{epoch}.pth"
            torch.save(checkpoint, checkpoint_path)

            # Save best model separately
            if is_best:
                best_model_path = self.save_dir / "best_unified_model.pth"
                torch.save(checkpoint, best_model_path)
                print(f"Best model saved to {best_model_path}")

            # Save latest model (always overwrite)
            latest_model_path = self.save_dir / "latest_unified_model.pth"
            torch.save(checkpoint, latest_model_path)

            # Build and save reference base every few epochs (live saving)
            if epoch % 5 == 0 or is_best:  # Save reference base every 5 epochs or when best
                print(f"Building and saving reference base at epoch {epoch}...")
                self.build_and_save_reference_base_live()

            print(f"Training state saved at epoch {epoch} (loss: {loss:.4f})")

        except Exception as e:
            print(f"Error saving training state: {e}")

    def build_and_save_reference_base_live(self):
        """Build and save reference base during training (live saving)"""
        try:
            # print("Building reference base from current model state...")
            self.model.eval()

            if len(self.dataset.segment_mapping) == 0:
                print("No segments available for building reference base!")
                return

            # Clear existing reference base
            self.reference_base = ReferenceBase(save_dir=str(self.save_dir))
            processed_pieces = set()

            with torch.no_grad():
                for data_id, segments in self.dataset.segment_mapping.items():
                    if data_id in processed_pieces:
                        continue

                    try:
                        # Get representative segments
                        symbolic_file = segments['symbolic'][0]  # Take first segment as representative
                        audio_file = segments['audio'][0]

                        # Load features with proper dimensions
                        symbolic_features = self.dataset._load_features(symbolic_file, expected_dim=768).to(self.device)
                        audio_features = self.dataset._load_features(audio_file, expected_dim=6144).to(self.device)

                        # Get unified embedding
                        unified_embedding = self.model(symbolic_features, audio_features)

                        # Add to reference base
                        metadata = {
                            'composer': segments['composer'],
                            'composition': segments['composition'],
                            'movement': segments.get('movement', ''),
                            'ensemble': segments.get('ensemble', ''),
                            'era': segments['era']
                        }

                        self.reference_base.add_reference(
                            data_id,
                            unified_embedding.cpu().numpy().flatten(),
                            metadata
                        )

                        processed_pieces.add(data_id)

                    except Exception as e:
                        print(f"Error processing piece {data_id}: {e}")
                        continue

            # Build searchable index
            self.reference_base.build_index()

            # Save reference base
            self.reference_base.save("music_reference_base.pkl")
            print(f"Reference base built and saved with {len(processed_pieces)} pieces")

        except Exception as e:
            print(f"Error building reference base: {e}")
        finally:
            self.model.train()  # Return to training mode

    def train_epoch(self):
        """Train for one epoch with better error handling"""
        self.model.train()
        total_loss = 0
        num_batches = 0
        successful_batches = 0

        for batch_idx, batch in enumerate(self.train_loader):
            try:
                symbolic_features = batch['symbolic'].to(self.device)
                audio_features = batch['audio'].to(self.device)
                labels = batch['label'].to(self.device)

                # Skip batch if it contains error data
                if 'error' in batch['pair_type']:
                    print(f"Skipping error batch {batch_idx}")
                    continue

                self.optimizer.zero_grad()

                # Get unified embeddings
                unified_embeddings = self.model(symbolic_features, audio_features)

                # Compute loss
                loss = self.criterion(unified_embeddings, labels)

                # Skip if loss is invalid
                if torch.isnan(loss) or torch.isinf(loss):
                    print(f"Skipping batch {batch_idx} due to invalid loss: {loss}")
                    continue

                # Backward pass
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()

                total_loss += loss.item()
                num_batches += 1
                successful_batches += 1

                # Print progress every 100 batches
                if batch_idx % 100 == 0:
                    print(f"Batch {batch_idx}/{len(self.train_loader)}, Loss: {loss.item():.4f}")

            except Exception as e:
                print(f"Error in batch {batch_idx}: {e}")
                continue

        if successful_batches == 0:
            print("WARNING: No successful batches in this epoch!")
            return float('inf')

        avg_loss = total_loss / successful_batches
        self.scheduler.step()
        return avg_loss

    def train(self, num_epochs: int = 50):
        """Train the enhanced model with live saving"""
        print(f"Training enhanced model on {len(self.dataset)} pairs")
        print(f"All models and reference base will be saved to: {self.save_dir}")
        print(f"Device: {self.device}")

        if len(self.dataset) == 0:
            print("ERROR: Dataset is empty, cannot train!")
            return

        # Load existing checkpoint if available
        latest_checkpoint = self.save_dir / "latest_unified_model.pth"
        start_epoch = 0

        if latest_checkpoint.exists():
            print(f"Found existing checkpoint, loading...")
            try:
                checkpoint = torch.load(latest_checkpoint, map_location=self.device)
                self.model.load_state_dict(checkpoint['model_state_dict'])
                self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                self.training_stats = checkpoint.get('training_stats', self.training_stats)
                start_epoch = checkpoint['epoch'] + 1
                print(f"Resumed training from epoch {start_epoch}")
            except Exception as e:
                print(f"Error loading checkpoint: {e}, starting from scratch")

        patience = 10
        patience_counter = 0

        for epoch in range(start_epoch, num_epochs):
            try:
                print(f"\n Epoch {epoch + 1}/{num_epochs}")
                avg_loss = self.train_epoch()

                if avg_loss == float('inf'):
                    print(f"Epoch {epoch + 1} failed, skipping...")
                    continue

                print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

                # Update training stats
                self.training_stats['epoch_losses'].append(avg_loss)
                self.training_stats['epochs_trained'] = epoch + 1

                # Check if this is the best model
                is_best = avg_loss < self.training_stats['best_loss']
                if is_best:
                    self.training_stats['best_loss'] = avg_loss
                    patience_counter = 0
                    print(f"New best model! Loss: {avg_loss:.4f}")
                else:
                    patience_counter += 1

                # Save training state (LIVE SAVING)
                self.save_training_state(epoch + 1, avg_loss, is_best=is_best)

                # Early stopping
                if patience_counter >= patience:
                    print(f"Early stopping after {patience} epochs without improvement")
                    break

            except Exception as e:
                print(f"Error in epoch {epoch + 1}: {e}")
                # Still try to save current state even if epoch failed
                try:
                    self.save_training_state(epoch + 1, float('inf'), is_best=False)
                except:
                    pass
                continue

        # Final save
        print(f"\n Training completed!")
        print(f"Total epochs: {self.training_stats['epochs_trained']}")
        print(f"Best loss: {self.training_stats['best_loss']:.4f}")

        # Final model and reference base save
        try:
            final_model_path = self.save_dir / "final_unified_plagiarism_model.pth"
            latest_checkpoint = self.save_dir / "latest_unified_model.pth"
            if latest_checkpoint.exists():
                import shutil
                shutil.copy2(latest_checkpoint, final_model_path)
                print(f"Final model saved to {final_model_path}")

            # Build final reference base
            print("Building final reference base...")
            self.build_and_save_reference_base_live()

        except Exception as e:
            print(f"Error in final save: {e}")

    def detect_plagiarism_unified(self,
                                symbolic_features: torch.Tensor,
                                audio_features: torch.Tensor,
                                threshold: float = 0.8) -> Dict:
        """Detect plagiarism using unified embeddings"""
        self.model.eval()

        with torch.no_grad():
            symbolic_features = symbolic_features.to(self.device)
            audio_features = audio_features.to(self.device)

            # Get unified embedding
            unified_embedding = self.model(symbolic_features, audio_features)

            # Search in reference base
            query_embedding = unified_embedding.cpu().numpy().flatten()
            similar_pieces = self.reference_base.search_similar(
                query_embedding,
                top_k=10,
                threshold=threshold
            )

            return {
                'unified_embedding': unified_embedding.cpu().numpy(),
                'similar_pieces': similar_pieces,
                'max_similarity': max([p['similarity'] for p in similar_pieces]) if similar_pieces else 0.0,
                'is_plagiarism': len(similar_pieces) > 0
            }

    def save_model(self, filename: str):
        """Save the trained model to the specified directory"""
        try:
            filepath = self.save_dir / filename
            torch.save({
                'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'scheduler_state_dict': self.scheduler.state_dict(),
                'training_stats': self.training_stats
            }, filepath)
            print(f"Model saved to {filepath}")
        except Exception as e:
            print(f"Error saving model to {filepath}: {e}")

    def load_model(self, filename: str):
        """Load a trained model from the specified directory"""
        try:
            filepath = self.save_dir / filename
            checkpoint = torch.load(filepath, map_location=self.device)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            if 'training_stats' in checkpoint:
                self.training_stats = checkpoint['training_stats']
            print(f"Model loaded from {filepath}")
        except Exception as e:
            print(f"Error loading model from {filepath}: {e}")


def validate_dataset_before_training(detector):
    """Validate dataset before starting training"""
    print("Validating dataset...")

    if len(detector.dataset) == 0:
        print("ERROR: Dataset is empty!")
        return False

    # Test loading a few samples
    print("Testing sample loading...")
    try:
        for i in range(min(5, len(detector.dataset))):
            sample = detector.dataset[i]
            print(f"Sample {i}: symbolic shape {sample['symbolic'].shape}, "
                  f"audio shape {sample['audio'].shape}, label {sample['label']}")
    except Exception as e:
        print(f"Error loading samples: {e}")
        return False

    # Test DataLoader
    print("Testing DataLoader...")
    try:
        batch = next(iter(detector.train_loader))
        print(f"Batch shapes - symbolic: {batch['symbolic'].shape}, "
              f"audio: {batch['audio'].shape}, labels: {batch['label'].shape}")
        return True
    except Exception as e:
        print(f"Error with DataLoader: {e}")
        return False


def main():
    """Main training and evaluation pipeline with ultra-fast loading and live saving"""
    base_dir = "/content/drive/MyDrive/MuseGuard"
    symbolic_dir = f"{base_dir}/extracted_features"      # MIDI embeddings
    audio_dir = f"{base_dir}/musicnet_embeddings"        # Audio/WAV embeddings
    metadata_csv = f"{base_dir}/musicnet_metadata.csv"
    save_dir = base_dir  # All outputs will be saved here

    print("=== ULTRA-FAST Music Plagiarism Detection System with LIVE SAVING ===")
    print()
    print("📁 Directory Setup:")
    print(f"  MIDI (symbolic) embeddings: {symbolic_dir}")
    print(f"  Audio (WAV) embeddings: {audio_dir}")
    print(f"  Metadata CSV: {metadata_csv}")
    print(f"  💾 SAVE DIRECTORY: {save_dir}")

    # Check if files exist
    for path, name in [(symbolic_dir, "MIDI/symbolic"), (audio_dir, "audio/WAV"), (metadata_csv, "metadata CSV")]:
        if not os.path.exists(path):
            print(f"ERROR: {name} path '{path}' does not exist!")
            return

    try:
        # Initialize enhanced detector with live saving
        print(f"\n Initializing ultra-fast music plagiarism detector with live saving...")
        print("Processing composers: Bach, Beethoven, Brahms, Cambini, Dvorak, Faure, Haydn, Mozart, Ravel, Schubert")

        detector = EnhancedMusicPlagiarismDetector(
            symbolic_dir=symbolic_dir,  # MIDI embeddings
            audio_dir=audio_dir,        # Audio embeddings
            metadata_csv=metadata_csv,
            save_dir=save_dir           # Save everything to Google Drive
        )

        # Validate dataset before training
        if not validate_dataset_before_training(detector):
            print("Dataset validation failed. Exiting...")
            return

        print(f"\n Dataset successfully created with {len(detector.dataset)} strategic pairs")
        print(" Pair types include:")
        print("  - Positive: Direct matches, overlapping segments, nearby segments")
        print("  - Negative: Same composer (hard), same era (medium), different era (easy)")

        # Train the model with live saving
        print(f"\n Starting training with unified embeddings and LIVE SAVING to {save_dir}...")

        detector.train(num_epochs=50)

        print(f"\n === Training completed successfully! ===")
        print(f"All files saved to: {save_dir}")
        print("Files available:")
        print("- final_unified_plagiarism_model.pth (final trained model)")
        print("- best_unified_model.pth (best performing model)")
        print("- latest_unified_model.pth (latest model state)")
        print("- music_reference_base.pkl (reference database)")
        print("- file_validation_cache.pkl (validation cache for future runs)")
        print("- checkpoint_epoch_X.pth (checkpoint files)")

    except Exception as e:
        print(f"Error during execution: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()

=== 🚀 ULTRA-FAST Music Plagiarism Detection System with LIVE SAVING ===
🎯 Performance Features:
  ✅ Fast file validation using size heuristics
  ✅ Lazy loading - only load files when needed
  ✅ Persistent validation cache
  ✅ Skip corrupted files automatically
  ✅ Batch error recovery
  ✅ Early stopping
  ✅ LIVE SAVING - Models and reference base saved during training!
  ✅ Resume training from interruptions
  ✅ Checkpoint recovery

📁 Directory Setup:
  MIDI (symbolic) embeddings: /content/drive/MyDrive/MuseGuard/extracted_features
  Audio (WAV) embeddings: /content/drive/MyDrive/MuseGuard/musicnet_embeddings
  Metadata CSV: /content/drive/MyDrive/MuseGuard/musicnet_metadata.csv
  💾 SAVE DIRECTORY: /content/drive/MyDrive/MuseGuard

🔄 Initializing ultra-fast music plagiarism detector with live saving...
🎼 Processing composers: Bach, Beethoven, Brahms, Cambini, Dvorak, Faure, Haydn, Mozart, Ravel, Schubert
Creating new validation cache
Loaded metadata for 330 pieces
Columns: ['id', 'compo